In [1]:
import cv2
from PIL import Image
import os
import numpy as np
import pandas as pd
from hashlib import md5
import ast
from scipy import ndimage

df = pd.read_csv('segmented.csv')

df = df.dropna()

old_df = None
if os.path.exists('validated.csv'):
    old_df = pd.read_csv('validated.csv')
    
# remove if already validated
if old_df is not None:
    df = df[~df['mask'].isin(old_df['mask'])]
    
invalid_df = None
if os.path.exists('invalid.csv'):
    invalid_df = pd.read_csv('invalid.csv')
    
# remove if already invalid
if invalid_df is not None:
    df = df[~df['mask'].isin(invalid_df['mask'])]

df

,image,x,y,mask
4553,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,250,202,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4554,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,318,166,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4555,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,293,202,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4556,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,69,324,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4557,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,291,180,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
...,...,...,...,...
5149,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,110,240,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
5150,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,21,327,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
5151,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,282,134,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...
5152,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,20,227,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...


In [2]:
cv2.namedWindow('image')


def get_display_image(row):
    cv2image = cv2.imread(row.image)

    mask = np.load(row['mask'])
    mask = mask.astype(np.uint8)
    
    # get contours
    #contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # draw contours
    #cv2.drawContours(cv2image, contours, -1, (255,0,0), 3)
    
    #return cv2image

    # apply the mask to the image
    mask_image = cv2.bitwise_and(cv2image, cv2image, mask=mask)
    
    # crop the image to the mask
    mask_image = mask_image[np.ix_(mask.any(1),mask.any(0))]
    
    return mask_image


image_index = 0
data = []
invalid = []

show_image = get_display_image(df.iloc[image_index])

while True:
    
    if show_image is None or show_image.shape[0] <= 0 or show_image.shape[1] <= 0:
        print(f'invalid mask {df.iloc[image_index]["mask"]}')

        row = df.iloc[image_index]
        invalid.append([row.image, row.x, row.y, row['mask']])

        image_index += 1
        if image_index >= len(df):
            break
        show_image = get_display_image(df.iloc[image_index])
        continue
        
    cv2.imshow("image", show_image)
    k = cv2.waitKey(50) & 0xFF
    
    # if escape key, exit
    if k == 27:
        break

    elif k == ord('n'):
        row = df.iloc[image_index]
        data.append([row.image, row.x, row.y, row['mask'], False])
        image_index += 1
        if image_index >= len(df):
            break
        
        show_image = get_display_image(df.iloc[image_index])
        
    elif k == ord('y'):
        row = df.iloc[image_index]
        data.append([row.image, row.x, row.y, row['mask'], True])
        image_index += 1
        if image_index >= len(df):
            break
        
        show_image = get_display_image(df.iloc[image_index])


# close all open windows
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jack/.local/share/virtualenvs/datadev-5ox7fytP/lib/python3.10/site-packages/cv2/qt/plugins"


invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_800_2400.144.78.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/20231202_112343_800_0.174.83.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_800_2400.205.80.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0045_800_2400.202.189.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/20231202_112343_800_0.185.128.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/20231202_112343_800_0.130.68.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0035_4400_1200.346.165.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0035_4400_1200.384.139.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0001_3600_1200.145.23.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0059_2000_2400.267.374.mask.npy
invalid mask /home/jack/Mounts/DiskOne/kona_coffee/masks/100_0047_160

In [3]:
df = pd.DataFrame(data, columns=['image', 'x', 'y', 'mask', 'valid'])

if old_df is not None:
    df = pd.concat([old_df, df])

df.to_csv('validated.csv', index=False)

df

,image,x,y,mask,valid
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,299,58,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,324,300,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,272,321,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,364,288,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,True
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,228,197,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...,False
...,...,...,...,...,...
491,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,110,240,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...,True
492,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,21,327,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...,True
493,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,282,134,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...,True
494,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,20,227,/home/jack/Mounts/DiskOne/kona_coffee/masks/IM...,True


In [6]:
df = pd.DataFrame(invalid, columns=['image', 'x', 'y', 'mask'])

old_df = None
if os.path.exists('invalid.csv'):
    old_df = pd.read_csv('invalid.csv')
    
# remove if already in old df
if old_df is not None:
    df = df[~df['mask'].isin(old_df['mask'])]

if old_df is not None:
    df = pd.concat([old_df, df])

df.to_csv('invalid.csv', index=False)

df

,image,x,y,mask
0,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,308,147,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
1,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,273,222,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
2,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,76,34,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
3,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,226,296,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
4,/home/jack/Mounts/DiskOne/kona_coffee/splits/1...,268,221,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
...,...,...,...,...
22,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,359,81,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
23,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,253,301,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
24,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,233,364,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
25,/home/jack/Mounts/DiskOne/kona_coffee/augmente...,147,360,/home/jack/Mounts/DiskOne/kona_coffee/masks/10...
